This progress bar provides functionality like tqdm and FastProgress, but:
- It doesn't use ipywidgets
- Its styling _just works_ (always fills the width)

_Unlike_ tqdm, `Progress` is strictly async: it doesn't run on a separate thread. That means for it to draw, your code needs to wait for some async operation. If your code doesn't do any actual async operations, you can use `asyncio.sleep(0)` to briefly allow it to draw. It will still only draw when it needs to, taking its internal rate limit into account.

There's no `update` method: to get it to update, you can either use it as an iterator, or write to its attributes. For example, setting the `count` or `metrics` schedules a redraw.

In [1]:
import asyncio
import time

from utils.progress import Progress


# As a context manager (auto-closes)
async with Progress(total=1000, description='context manager') as p:
    for i in range(1001):
        p.count += 1
        if i % 100 == 0:
            p.mark(f'{i:d}')
        await asyncio.sleep(0.001)

# As an iterator
async for _ in Progress(range(1000), description='iterator'):
    await asyncio.sleep(0.001)

# As an iterator with assignment, to allow setting other attributes
async for i in (p := Progress(range(1000), description=('assigned iter'))):
    x = i + 1
    p.metrics |= {'x**2': x**2}
    await asyncio.sleep(0.001)

# Fully manual
p = Progress(total=1000, description='manual')
for _ in range(1000):
    p.count += 1
    await asyncio.sleep(0.001)
p.close()

context manager: 100.1% [1001/1000]

iterator: 100.0% [1000/1000]

assigned iter: 100.0% [1000/1000]

manual: 100.0% [1000/1000]

## Automatic task cooperation

If you don't care when other async tasks run, you can use the `co_op` utility. It wraps an iterable (such as `Progress`), calling `asyncio.sleep(0)` once per iteration. You still need to use `async for`, but you don't need to do other async operations.

In [2]:
import time

from utils.progress import Progress
from utils.progress.iterators import co_op

# Automatically allow other tasks to run.
async for _ in co_op(Progress(range(1000), description='task co-op')):
    # This sleep statement isn't async, so it does _not_ allow other tasks to run.
    # It's just here to simulate work.
    time.sleep(0.001)

task co-op: 100.0% [1000/1000]

## Synchronous version

In [ ]:
import time
from utils.progress import SyncProgress

# As an iterator
for _ in SyncProgress(range(1000), description='sync iterator'):
    time.sleep(0.001)

# As a context manager (auto-closes)
with SyncProgress(total=1000, description='sync context manager') as p:
    for _ in range(1000):
        p.count += 1
        time.sleep(0.001)

sync iterator: 100.0% [1000/1000]

sync context manager: 100.0% [1000/1000]